# Data Manipulation with Pandas

<div class="alert alert-block alert-danger">
<b>Check the Kernel you are using:</b> Before we get started, if you are running this on HiPerGator, double check the kernel in use. This is shown in the top right of the window and should look like: <img src="images/kernel.python310.png" alt"Image showing that the notebook is using the Python 3.10 kernel" style="float:right">
</div>

This notebook is based on [chapter 3 of Jake VanderPlas' Python Data Science Handbook](https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html). [<img src="images/PDSH-cover-small.png" alt="PDSH Cover Image" style="width: 50px;float:right"/>](https://jakevdp.github.io/PythonDataScienceHandbook/03.00-introduction-to-pandas.html)

As the chapter points out, Pandas builds on NumPy. We saw how NumPy has great features for fast, memory efficient numerical computations. But if you think about data, it is often organized in tables. NumPy has no functionality to label columns or rows with meaningful names. Reffering to managing data, which often include missing values and non-conforming data types, the text notes:
> Pandas, and in particular its `Series` and `DataFrame` objects, builds on the NumPy array structure and provides efficient access to these sorts of "data munging" tasks that occupy much of a data scientist's time.

Let's load NumPy and Pandas, using the standard alias `pd` and get the version we have:

In [1]:
import numpy as np
import pandas as pd
pd.__version__

'2.0.3'

The text mentions the three main data types that Pandas provides: `Series`, `DataFrame` and `Index`. We'll stat with `Series` as it helps connect NumPy arrays with Pandas objects.

## Pandas `Series`

In [2]:
data = pd.Series([0.25,0.5,0.75,1])
data

0    0.25
1    0.50
2    0.75
3    1.00
dtype: float64

> As we see in the output, the `Series` wraps both a sequence of values and a sequence of indices, which we can access with the `values` and `index` attributes. The `values` are simply a familiar NumPy array:

In [3]:
data.values

array([0.25, 0.5 , 0.75, 1.  ])

In [5]:
data.index

RangeIndex(start=0, stop=4, step=1)

> The essential difference \[between a NymPy array and a Pandas `Series`\] is the presence of the index: while the Numpy Array has an *implicitly* defined integer index used to access the values, the Pandas `Series` has an *explicitly* defined index associated with the values.

In [8]:
# Indices do not need to be integers:

data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

As the text points out, this is similar to a Python dictionary. But with Pandas `Series` both the index and the values are typed. As with NymPy arrays, the explicit typing in `Series` add efficiency in memmory and computation. 

## Pandas `DataFrame`

The main data structre that makes Pandas popular is the `DataFrame` class. 

> If a `Series` is an analog of a one-dimensional array with flexible indices, a `DataFrame` is an analog of a two-dimensional array with both flexible row indices and flexible column names.

Here's the example of two `Series` that the text uses to combine into a `DataFrame`:

In [9]:
population_dict = {'California': 38332521,
                   'Texas': 26448193,
                   'New York': 19651127,
                   'Florida': 19552860,
                   'Illinois': 12882135}
population = pd.Series(population_dict)
population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
dtype: int64

In [10]:
area_dict = {'California': 423967, 'Texas': 695662, 'New York': 141297,
             'Florida': 170312, 'Illinois': 149995}
area = pd.Series(area_dict)
area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
dtype: int64

In [12]:
states = pd.DataFrame({'population': population, 'area': area})
states

,population,area
California,38332521,423967
Texas,26448193,695662
New York,19651127,141297
Florida,19552860,170312
Illinois,12882135,149995


In [18]:
# In addition to the index, DataFrames have columns
print(states.index)

print(states.columns)

type(states)

Index(['California', 'Texas', 'New York', 'Florida', 'Illinois'], dtype='object')
Index(['population', 'area'], dtype='object')


pandas.core.frame.DataFrame

## Accessing data in `DataFrame`s

In [19]:
states['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [20]:
 # Careful: Using attributes can break
states.area

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

In [23]:
# Add a new column
states.density = states.population/states.area
states

,population,area,density
California,38332521,423967,90.413926
Texas,26448193,695662,38.018740
New York,19651127,141297,139.076746
Florida,19552860,170312,114.806121
Illinois,12882135,149995,85.883763


In [33]:
states.population['California']

38332521

In [40]:
states.iloc[0,1]

423967

In [35]:
states.California

AttributeError: 'DataFrame' object has no attribute 'California'

In [34]:
states.loc['California']['population']

38332521.0

## Index alignment

I'm skipping a bunch of stuff in the text, but want to show this important and useful feature of `Series`.

> For binary operations on two `Series` or `DataFrame` objects, Pandas will align indices in the process of performing the operation. This is very convenient when working with incomplete data, as we'll see in some of the examples that follow.

In [24]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

In [26]:
population/area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

In [30]:
'tEXAS'.capitalize()

'Texas'

In [25]:
states_1 = pd.DataFrame({'population': population, 'area': area})
states_1 

,population,area
Alaska,NaN,1723337.0
California,38332521.0,423967.0
New York,19651127.0,NaN
Texas,26448193.0,695662.0


Notice how Pandas seamlessly manages the combination of these partially overlapping `Series` indices.

## Handling Missing Data

> The difference between data found in many tutorials and data in the real world is that real-world data is rarely clean and homogeneous. In particular, many interesting datasets will have some amount of data missing. To make matters even more complicated, different data sources may indicate missing data in different ways.

> There are a number of schemes that have been developed to indicate the presence of missing data in a table or DataFrame. Generally, they revolve around one of two strategies: using a **mask** that globally indicates missing values, or choosing a **sentinel** value that indicates a missing entry.

### `None`: Pythonic missing data

> The first sentinel value used by Pandas is `None`, a Python singleton object that is often used for missing data in Python code. Because it is a Python object, `None` cannot be used in any arbitrary NumPy/Pandas array, but only in arrays with data type `'object'` (i.e., arrays of Python objects):

In [45]:
vals1=np.array([1,None,3,4])
vals1
vals1.sum()

TypeError: unsupported operand type(s) for +: 'int' and 'NoneType'

### `NaN`: Missing numerical data

> The other missing data representation, `NaN` (acronym for Not a Number), is different; it is a special floating-point value recognized by all systems that use the standard IEEE floating-point representation:

In [44]:
vals2=np.array([1,np.nan,3,4])
vals2
vals2.sum()

nan

But be a bit careful...`NaN` "is a bit like a data virus–it infects any other object it touches."

In [46]:
# There are some NaN aware functions
np.nansum(vals2)
np.nanmin(vals2)
np.nanmax(vals2)


4.0

> Keep in mind that `NaN` is specifically a floating-point value; there is no equivalent `NaN` value for integers, strings, or other types.

## Dealing with null values

In [53]:
data = pd.Series([1, np.nan, 'hello', None])
data

0        1
1      NaN
2    hello
3     None
dtype: object

In [54]:
# Detecting null values
data
data[data.notnull()]

0        1
2    hello
dtype: object

In [ ]:
# Use a boolean mask to get non-null values


In [57]:
# Or more simply dropna
# this creates a copy rather than modify the dataset
data.dropna()

0        1
2    hello
dtype: object

In [58]:
df = pd.DataFrame([[1,      np.nan, 2],
                   [2,      3,      5],
                   [np.nan, 4,      6]])
df

,0,1,2
0,1.0,NaN,2
1,2.0,3.0,5
2,NaN,4.0,6


In [60]:
# dropna by default drops rows with NaN
df.dropna()

,0,1,2
1,2.0,3.0,5


In [61]:
# Or use axis= 1 or 'columns'
df.dropna(axis='columns')

,2
0,2
1,5
2,6


In [ ]:
# There is a how but thresh is probably more usefull
# Minimum number of non NaN values to keep



## Filling Null Values

> Sometimes rather than dropping `NA` values, you'd rather replace them with a valid value. This value might be a single number like zero, or it might be some sort of imputation or interpolation from the good values. You could do this in-place using the `isnull()` method as a mask, but because it is such a common operation Pandas provides the `fillna()` method, which returns a copy of the array with the null values replaced.

In [ ]:
data = pd.Series([1, np.nan, 2, None, 3], index=list('abcde'))
data

In [ ]:
# forward-fill


In [ ]:
# back-fill


In [ ]:
# Similar in DataFrames

df[3] = np.nan # I skipped a step earlier where the text added a column of NaNs
df